In [1]:
import sys
sys.path.append("../")
from mir.common.parser import *
from mir.common.repertoire import Repertoire, RepertoireDataset
from mir.basic.segment_usage import *
from mir.basic.sampling import RepertoireSampling
import time

In [39]:
import importlib
imported_module = importlib.import_module("mir.common.repertoire")
importlib.reload(imported_module)

from mir.common.repertoire import Repertoire, RepertoireDataset

In [3]:
from sklearn.manifold import TSNE

In [4]:
metadata = pd.read_csv('assets/samples/fmba_healthy/metadata.csv')
metadata

,sequencingDate,folder,R1,R2,name,COVID_status,projectName,COVID_IgG,COVID_IgM,COVID_PCR,...,HLA-DQB1.2,HLA-DRB1.1,HLA-DRB1.2,id,subsequent_sequencing,HLA-DRB4.1,HLA-DRB3.1,HLA-DRB5.1,gene,run
0,2020 10 19,2020/10_FMBA_NovaSeq6,770101520808_S102_L002_R1_001.fastq.gz,770101520808_S102_L002_R2_001.fastq.gz,770101520808,healthy,FMBA_DNA,no,no,negative,...,DQB1*06:02,DRB1*11:04,DRB1*15:01,770101520,NaN,NaN,NaN,NaN,NaN,770101520808_S102_L002.clonotypes.TRB.txt
1,2020 10 19,2020/10_FMBA_NovaSeq6,500001210807_S66_L001_R1_001.fastq.gz,500001210807_S66_L001_R2_001.fastq.gz,500001210807,healthy,FMBA_DNA,no,no,negative,...,DQB1*04:02,DRB1*07:01,DRB1*08:01,500001210,NaN,NaN,NaN,NaN,NaN,500001210807_S66_L001.clonotypes.TRB.txt
2,2020 10 19,2020/10_FMBA_NovaSeq6,500002190807_S8_L001_R1_001.fastq.gz,500002190807_S8_L001_R2_001.fastq.gz,500002190807,healthy,FMBA_DNA,no,no,negative,...,DQB1*04:02,DRB1*07:01,DRB1*08:01,500002190,NaN,NaN,NaN,NaN,NaN,500002190807_S8_L001.clonotypes.TRB.txt
3,2020 10 19,2020/10_FMBA_NovaSeq4,780003510808_S92_L001_R1_001.fastq.gz,780003510808_S92_L001_R2_001.fastq.gz,780003510808,healthy,FMBA_DNA,no,no,negative,...,DQB1*03:03,DRB1*07:01,DRB1*07:01,780003510,NaN,NaN,NaN,NaN,NaN,780003510808_S92_L001.clonotypes.TRB.txt
4,2020 10 19,2020/10_FMBA_NovaSeq4,550001610808_S148_L002_R1_001.fastq.gz,550001610808_S148_L002_R2_001.fastq.gz,550001610808,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:01,DRB1*01:01,DRB1*11:04,550001610,NaN,NaN,NaN,NaN,NaN,550001610808_S148_L002.clonotypes.TRB.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020 10 19,2020/10_FMBA_NovaSeq5,270000140808_S174_L002_R1_001.fastq.gz,270000140808_S174_L002_R2_001.fastq.gz,270000140808,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:02,DRB1*16:01,DRB1*16:01,270000140,NaN,NaN,NaN,NaN,NaN,270000140808_S174_L002.clonotypes.TRB.txt
96,2020 10 19,2020/10_FMBA_NovaSeq5,500001400808_S155_L002_R1_001.fastq.gz,500001400808_S155_L002_R2_001.fastq.gz,500001400808,healthy,FMBA_DNA,no,no,negative,...,DQB1*06:01,DRB1*07:01,DRB1*15:02,500001400,NaN,NaN,NaN,NaN,NaN,500001400808_S155_L002.clonotypes.TRB.txt
97,2020 10 19,2020/10_FMBA_NovaSeq6,780004000807_S5_L001_R1_001.fastq.gz,780004000807_S5_L001_R2_001.fastq.gz,780004000807,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:01,DRB1*01:01,DRB1*04:01,780004000,NaN,NaN,NaN,NaN,NaN,780004000807_S5_L001.clonotypes.TRB.txt
98,2020 10 19,2020/10_FMBA_NovaSeq4,550001850808_S122_L002_R1_001.fastq.gz,550001850808_S122_L002_R2_001.fastq.gz,550001850808,healthy,FMBA_DNA,no,no,negative,...,DQB1*05:01,DRB1*01:01,DRB1*04:01,550001850,NaN,NaN,NaN,NaN,NaN,550001850808_S122_L002.clonotypes.TRB.txt


In [40]:
t0 = time.time()
dataset = RepertoireDataset.load(parser=VDJtoolsParser(sep=','), 
                                 metadata=metadata,
                                 threads=28,
                                 paths=['assets/samples/fmba_healthy/' + r['run'] + '.gz'
                                        for _, r in metadata.iterrows()])
print(time.time() - t0)


69.41856861114502


In [41]:
folder_to_run_mapping = {}
for folder in dataset.metadata[['run', 'folder']].folder.unique():
    folder_to_run_mapping[folder] = set(dataset.metadata[dataset.metadata.folder == folder].run)

In [42]:
norm_usage_table_v = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V')
norm_usage_table_j = NormalizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J')

z_score_usage_table_v = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='V', group_mapping=folder_to_run_mapping, standardization_method='log_exp')
z_score_usage_table_j = StandardizedSegmentUsageTable.load_from_repertoire_dataset(repertoire_dataset=dataset, gene='TRB', segment_type='J', group_mapping=folder_to_run_mapping, standardization_method='log_exp')


In [43]:
dataset[0].number_of_reads

7823059

## Sampling reads from the repertoire, not changing the gene usage distribution

In [44]:
sampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=None, n=100000)

In [45]:
sampled_repertoire

Repertoire of 14942 clonotypes and 100000 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSFWTGQLNTEAFF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSSPG_CQPQHF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSPDSQSSGNTIYF cdr3nt_2
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSVVGAQSGANVLTF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYSESEAFF cdr3nt_4
sequencingDate                                                  2020 10 19
folder                                               2020/10_FMBA_NovaSeq6
R1                                  7701015208

## Resampling the repertoire using the new derived usages for V and J genes

In [46]:
resampled_repertoire = RepertoireSampling().sample(repertoire=dataset[0], old_usage_matrix=[norm_usage_table_v, norm_usage_table_j], new_usage_matrix=[z_score_usage_table_v, z_score_usage_table_j], n=100000)

In [47]:
resampled_repertoire

Repertoire of 14342 clonotypes and 100000 cells:
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSFWTGQLNTEAFF cdr3nt_0
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSSPG_CQPQHF cdr3nt_1
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSVVGAQSGANVLTF cdr3nt_3
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CAISDPGGNSPLHF cdr3nt_8
κIndex(['count', 'freq', 'cdr3nt', 'cdr3aa', 'v', 'd', 'j', 'VEnd', 'DStart',
       'DEnd', 'JStart'],
      dtype='object') CASSYSKATRR_GNLSGANVLTF cdr3nt_5
sequencingDate                                                  2020 10 19
folder                                               2020/10_FMBA_NovaSeq6
R1                                  7

## Resampling all the reperoires from a dataset

In [ ]:
# t0 = time.time()
# resampled_dataset = RepertoireDataset.resample(initial_segment_usage_tables=[norm_usage_table_v, norm_usage_table_j], updated_segment_usage_tables=[z_score_usage_table_v, z_score_usage_table_j], n=100000, threads=28)
# print(time.time() - t0)
